In [1]:
import pandas as pd

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df

,Lugar de residencia,Medida,Valor
0,Huelva,Población,510743
1,Huelva,Confirmados PDIA,10199
2,Huelva,Confirmados PDIA 14 días,1331
3,Huelva,Tasa PDIA 14 días,"260,6007326581079"
4,Huelva,Confirmados PDIA 7 días,564
...,...,...,...
627,Municipio de Huelva sin especificar,Tasa PDIA 14 días,-
628,Municipio de Huelva sin especificar,Confirmados PDIA 7 días,3
629,Municipio de Huelva sin especificar,Total Confirmados,124
630,Municipio de Huelva sin especificar,Curados,88


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  10199.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  2863.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 951 personas en los últimos 7 días 

Un positivo PCR cada 378 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,10199.0,564.0,1331.0,510743.0,110.427358,260.600733,222.0
Huelva-Costa,5683.0,375.0,886.0,288115.0,130.156361,307.516096,163.0
Huelva (capital),2863.0,151.0,380.0,143663.0,105.107091,264.507911,65.0
Condado-Campiña,3129.0,155.0,366.0,155057.0,99.963239,236.042230,46.0
Isla Cristina,375.0,78.0,121.0,21264.0,366.817156,569.036870,42.0
Cartaya,346.0,39.0,74.0,19974.0,195.253830,370.481626,26.0
Lepe,901.0,56.0,139.0,27431.0,204.148591,506.725967,20.0
Palma del Condado (La),467.0,34.0,70.0,10761.0,315.955766,650.497166,15.0
Sierra de Huelva-Andévalo Central,1264.0,31.0,63.0,67571.0,45.877669,93.235264,11.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Lucena del Puerto,82.0,8.0,27.0,3371.0,237.318303,800.949273,0.0
Palma del Condado (La),467.0,34.0,70.0,10761.0,315.955766,650.497166,15.0
Isla Cristina,375.0,78.0,121.0,21264.0,366.817156,569.036870,42.0
San Bartolomé de la Torre,44.0,13.0,19.0,3736.0,347.965739,508.565310,3.0
Lepe,901.0,56.0,139.0,27431.0,204.148591,506.725967,20.0
Sanlúcar de Guadiana,4.0,1.0,2.0,409.0,244.498778,488.997555,NaN
Villanueva de los Castillejos,58.0,4.0,13.0,2820.0,141.843972,460.992908,0.0
Santa Ana la Real,3.0,2.0,2.0,476.0,420.168067,420.168067,0.0
Villalba del Alcor,100.0,7.0,14.0,3338.0,209.706411,419.412822,2.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Nerva,87.0,1.0,9.0,5235.0,19.102197,171.919771,1.0,0.111111
Ayamonte,317.0,10.0,69.0,20946.0,47.741812,329.418505,0.0,0.144928
Aroche,52.0,2.0,8.0,3073.0,65.082981,260.331923,1.0,0.250000
Bollullos Par del Condado,232.0,2.0,8.0,14272.0,14.013453,56.053812,1.0,0.250000
Cerro de Andévalo (El),37.0,1.0,4.0,2364.0,42.301184,169.204738,1.0,0.250000
Lucena del Puerto,82.0,8.0,27.0,3371.0,237.318303,800.949273,0.0,0.296296
Rociana del Condado,114.0,6.0,20.0,7855.0,76.384468,254.614895,0.0,0.300000
Palos de la Frontera,214.0,14.0,46.0,11289.0,124.014527,407.476304,8.0,0.304348
Villanueva de los Castillejos,58.0,4.0,13.0,2820.0,141.843972,460.992908,0.0,0.307692
